In [ ]:
from Classifications import Classifications
import tensorflow as tf
from tensorflow.keras.layers import InputLayer, Conv1D, GlobalAveragePooling1D, Dense, Lambda
from tensorflow.keras.models import Sequential

In [ ]:
TIME_STEPS = 48
NUM_FEATURES = 6
classifications = Classifications()
model_h_path = "../model_data.h"

In [ ]:
slice_indexes = tf.cumsum(tf.concat([[0], [len(v) for v in classifications.classifications.values()]], axis=0))

def custom_softmax(x):
    return tf.concat([
        tf.nn.softmax(x[:, slice_indexes[i]:slice_indexes[i+1]]) 
        for i in range(len(slice_indexes) - 1)
    ], axis=1)

model = Sequential([
    InputLayer(shape=(TIME_STEPS, NUM_FEATURES)),
    Conv1D(64, kernel_size=3, activation="relu"),
    GlobalAveragePooling1D(),
    Dense(32, activation="relu"),
    Dense(16, activation="relu"),
    Dense(classifications.num_classes),
    Lambda(custom_softmax),
])

model.summary()

In [ ]:
sample_input = tf.random.uniform(shape=(1, TIME_STEPS, NUM_FEATURES))
prediction = model.predict(sample_input)

start_idx = end_idx = 0
predictions = []
for classes in classifications.classifications.values():
    end_idx += len(classes)
    pred_idx = tf.argmax(prediction[:, start_idx:end_idx], axis=1)[0]
    start_idx = end_idx
    predictions.append(classes[pred_idx])
print(f"Prediction: {' '.join(reversed(predictions))}")

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

In [ ]:
with open(model_h_path, "w") as f:
    f.write("#ifndef _MODEL_DATA_H_\n#define _MODEL_DATA_H_\n")
    f.write("const unsigned char model[] = {")
    f.write(",".join(f"0x{b:02x}" for b in tflite_model))
    f.write("};\n#endif\n")